In [4]:
import requests
try:
    import cookielib
except:
    import http.cookiejar as cookielib
import re
import time
import os.path
import json
from bs4 import BeautifulSoup
try:
    from PIL import Image
except:
    pass

# from mywordCloud import save_jieba_result
# from mywordCloud import draw_wordcloud
import threading
import codecs
# 构造 Request headers
agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
headers = {
    "Host": "www.douban.com",
    "Referer": "https://www.douban.com/",
    'User-Agent': agent,
}

#使用cookie登录信息
session=requests.session()
session.cookies=cookielib.LWPCookieJar(filename='cook')

try:
    session.cookies.load(ignore_discard=True)
    print('成功加载cookie')
except:
    print("cookie 未能加载")

# 获取验证码
def get_captcha(url):
    #获取验证码
    print('获取验证码',url)
    captcha_url = url
    r = session.get(captcha_url, headers=headers)
    print('test')
    with open('captcha.jpg', 'wb') as f:
        f.write(r.content)
        f.close()
    # 用pillow 的 Image 显示验证码
    # 如果没有安装 pillow 到源代码所在的目录去找到验证码然后手动输入
    try:
        im = Image.open('captcha.jpg')
        im.show()
        im.close()
    except:
        print(u'请到 %s 目录找到captcha.jpg 手动输入' % os.path.abspath('captcha.jpg'))
    captcha = input("please input the captcha\n>")
    return captcha

def isLogin():
    #登录个人主页，查看是否登录成功
    url='https://www.douban.com/people/151607908/'
    login_code=session.get(url,headers=headers,allow_redirects=False).status_code
    if login_code==200:
        return True
    else:
        return False


def login(acount,secret):
    douban="https://www.douban.com/"
    htmlcha=session.get(douban,headers=headers).text
    patterncha=r'id="captcha_image" src="(.*?)" alt="captcha"'
    httpcha=re.findall(patterncha,htmlcha)
    pattern2=r'type="hidden" name="captcha-id" value="(.*?)"'
    hidden_value=re.findall(pattern2,htmlcha)
    print(hidden_value)

    post_data = {
        "source": "index_nav",
        'form_email': acount,
        'form_password': secret
    }
    if len(httpcha)>0:
        print('验证码连接',httpcha)
        capcha=get_captcha(httpcha[0])
        post_data['captcha-solution']=capcha
        post_data['captcha-id']=hidden_value[0]

    print (post_data)
    post_url='https://www.douban.com/accounts/login'
    login_page=session.post(post_url,data=post_data,headers=headers)
    #保存cookies
    session.cookies.save()

    if isLogin():
        print('登录成功')
    else:
        print('登录失败')

#豆瓣评分等级
gradeDic = {
    '力荐':5,
    '推荐':4,
    '还行':3,
    '较差':2,
    '很差':1
}
        
def get_movie_sort():
    time.sleep(1)
    movie_url='https://movie.douban.com/chart'
    html=session.get(movie_url,headers=headers)
    soup=BeautifulSoup(html.text,'html.parser')
    result=soup.find_all('a',{'class':'nbg'})
    print(result)

# #爬取短评论
# def get_comment(filename):  #filename为爬取得内容保存的文件
#     begin=1
#     next_url='?start=20&limit=20&sort=new_score&status=P'
#     f=open(filename,'w+',encoding='utf-8')
#     while(True):
#         time.sleep(5)
#         comment_url='https://movie.douban.com/subject/26363254/comments'
#         data={
#             'start':'27',
#             'limit':'-20',
#             'sort':'new_score',
#             'status':'P'
#         }
#         headers2 = {
#             "Host": "movie.douban.com",
#             "Referer": "https://www.douban.com/",
#             'User-Agent': agent,
#             'Connection': 'keep-alive',
#         }

#         html=session.get(url='https://movie.douban.com/subject/26363254/comments'+next_url,headers=headers2)
#         soup=BeautifulSoup(html.text,'html.parser')

#         #爬取当前页面的所有评论
#         result=soup.find_all('div',{'class':'comment'}) #爬取得所有的短评
#         pattern4 = r'<p class=""> (.*?)' \
#                    r'</p>'
#         for item in result:
#             s=str(item)
#             count2=s.find('<p class="">')
#             count3=s.find('</p>')
#             s2=s[count2+12:count3]  #抽取字符串中的评论
#             if 'class' not in s2:
#                 f.write(s2)

#         #获取下一页的链接
#         next_url=soup.find_all('div',{'id':'paginator'})
#         pattern3=r'href="(.*?)">后页'
#         if(len(next_url)==0):
#             break
#         next_url=re.findall(pattern3,str(next_url[0]))  #得到后页的链接
#         if(len(next_url)==0): #如果没有后页的链接跳出循环
#             break
#         next_url=next_url[0]
#         print('%d爬取下一页评论...'%begin)
#         begin=begin+1
#         #如果爬取了6次则多休息2秒
#         if(begin%6==0):
#             time.sleep(30)
#             print('休息...')
#         print(next_url)
#     f.close()

#多线程爬虫，爬取豆瓣影评
def thread_get_comment(comment_url):
    next_url = '?start=19&limit=20&sort=new_score&status=P'
    headers2 = {
        "Host": "movie.douban.com",
        "Referer": "https://www.douban.com/",
        'User-Agent': agent,
        'Connection': 'keep-alive',
    }
    filename='key.txt'
    f = open(filename, 'w+', encoding='utf-8')
    comment_url = 'https://movie.douban.com/subject/26363254/comments'
    crawl_queue=[comment_url+next_url]
    crawl_queue.append('https://movie.douban.com/subject/26363254/comments?start=144&limit=20&sort=new_score&status=P')
    seen=set(crawl_queue)
    seen=set(crawl_queue)

    def process_queue():
        begin = 1
        while True:
            try:
                url=crawl_queue.pop()
            except  IndexError:
                break
            else:
                time.sleep(20)
                html = session.get(url=url,headers=headers2)
                soup = BeautifulSoup(html.text, 'html.parser')

                # 爬取当前页面的所有评论
                result = soup.find_all('div', {'class': 'comment'})  # 爬取得所有的短评
                pattern4 = r'<p class=""> (.*?)' \
                           r'</p>'
                for item in result:
                    s = str(item)
                    count2 = s.find('<p class="">')
                    count3 = s.find('</p>')
                    s2 = s[count2 + 12:count3]  # 抽取字符串中的评论
                    if 'class' not in s2:
                        f.write(s2)

                # 获取下一页的链接
                next_url = soup.find_all('div', {'id': 'paginator'})
                pattern3 = r'href="(.*?)">后页'
                if (len(next_url) == 0):
                    break
                next_url = re.findall(pattern3, str(next_url[0]))  # 得到后页的链接
                if (len(next_url) == 0):  # 如果没有后页的链接跳出循环
                    break
                next_url = next_url[0]
                print('%d爬取下一页评论...' % begin)
                begin = begin + 1
                # 如果爬取了6次则多休息2秒
                if (begin % 6 == 0):
                    print('休息...')
                    time.sleep(30)

                print(next_url)
                if comment_url+next_url not in seen:
                    seen.add(comment_url+next_url)
                    crawl_queue.append(comment_url+next_url)

    threads=[]
    max_threads=5
    while threads or crawl_queue:
        for thread in threads:
            if not thread.is_alive():
                threads.remove(thread)
        while len(threads)< max_threads and crawl_queue:
            thread=threading.Thread(target=process_queue)
            print('------------------------下一个线程----------')
            thread.setDaemon(True) # set daemon so main thread can exit when receive ctrl + C
            thread.start()
            threads.append(thread)
        time.sleep(2)


    f.close()

if __name__=='__main__':
    if isLogin():
        print('您已经登录')
    else:
        print('xs')
        #login('name','psd')
        login('15300523175','@Zy107868')
    #get_comment('key.txt')  #单线程爬虫
   
    thread_get_comment('https://movie.douban.com/subject/25823277/collections')  #多线程爬虫
    #save_jieba_result('key2.txt')
    #draw_wordcloud('pjl_jieba.txt')



成功加载cookie
xs
[]
{'source': 'index_nav', 'form_email': '15300523175', 'form_password': '@Zy107868'}
登录成功
------------------------下一个线程----------
------------------------下一个线程----------
1爬取下一页评论...
?start=40&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
1爬取下一页评论...
?start=164&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
2爬取下一页评论...
2爬取下一页评论...?start=184&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=

?start=60&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
3爬取下一页评论...
?start=204&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
3爬取下一页评论...
?start=80&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
4爬取下一页评论...
?start=224&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
4爬取下一页评论...
?start=100&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
5爬取下一页评论...
休息...
5爬取下一页评论...
休息...
?start=244&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
?start=120&amp;limit

成功加载cookie
xs
[]
{'source': 'index_nav', 'form_email': '_name', 'form_password': 'psd'}
登录失败
------------------------下一个线程----------
https://movie.douban.com/subject/25823277/collections?start=4480
224爬取下一页评论...


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\admin\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-ef754ff59704>", line 214, in process_queue
    info = comment.find('span',class_='comment-info')  # 获取用户和评分信息栏
TypeError: find() takes no keyword arguments



https://movie.douban.com/subject/25823277/collections?start=4500
225爬取下一页评论...
https://movie.douban.com/subject/25823277/collections?start=4520
226爬取下一页评论...


In [31]:
import requests
try:
    import cookielib
except:
    import http.cookiejar as cookielib
import re
import csv
import time
import os.path
import json
from bs4 import BeautifulSoup
try:
    from PIL import Image
except:
    pass

# from mywordCloud import save_jieba_result
# from mywordCloud import draw_wordcloud
import threading
import codecs
# 构造 Request headers
agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
headers = {
    "Host": "www.douban.com",
    "Referer": "https://www.douban.com/",
    'User-Agent': agent,
}

#使用cookie登录信息
session=requests.session()
session.cookies=cookielib.LWPCookieJar(filename='cook')

try:
    session.cookies.load(ignore_discard=True)
    print('成功加载cookie')
except:
    print("cookie 未能加载")

# 获取验证码
def get_captcha(url):
    #获取验证码
    print('获取验证码',url)
    captcha_url = url
    r = session.get(captcha_url, headers=headers)
    print('test')
    with open('captcha.jpg', 'wb') as f:
        f.write(r.content)
        f.close()
    # 用pillow 的 Image 显示验证码
    # 如果没有安装 pillow 到源代码所在的目录去找到验证码然后手动输入
    try:
        im = Image.open('captcha.jpg')
        im.show()
        im.close()
    except:
        print(u'请到 %s 目录找到captcha.jpg 手动输入' % os.path.abspath('captcha.jpg'))
    captcha = input("please input the captcha\n>")
    return captcha

def isLogin():
    #登录个人主页，查看是否登录成功
    url='https://www.douban.com/people/151607908/'
    login_code=session.get(url,headers=headers,allow_redirects=False).status_code
    if login_code==200:
        return True
    else:
        return False

#将内容存入excel
def write_data(datas, name):
    file_name = name
    with open(file_name, 'a', errors='ignore', newline='') as f:
            f_csv = csv.writer(f)
            f_csv.writerows(datas)

            
def login(acount,secret):
    douban="https://www.douban.com/"
    htmlcha=session.get(douban,headers=headers).text
    patterncha=r'id="captcha_image" src="(.*?)" alt="captcha"'
    httpcha=re.findall(patterncha,htmlcha)
    pattern2=r'type="hidden" name="captcha-id" value="(.*?)"'
    hidden_value=re.findall(pattern2,htmlcha)
    print(hidden_value)

    post_data = {
        "source": "index_nav",
        'form_email': acount,
        'form_password': secret
    }
    if len(httpcha)>0:
        print('验证码连接',httpcha)
        capcha=get_captcha(httpcha[0])
        post_data['captcha-solution']=capcha
        post_data['captcha-id']=hidden_value[0]

    print (post_data)
    post_url='https://www.douban.com/accounts/login'
    login_page=session.post(post_url,data=post_data,headers=headers)
    #保存cookies
    session.cookies.save()

    if isLogin():
        print('登录成功')
    else:
        print('登录失败')

#豆瓣评分等级
gradeDic = {
    '力荐':5,
    '推荐':4,
    '还行':3,
    '较差':2,
    '很差':1
}
        
def get_movie_sort():
    time.sleep(1)
    movie_url='https://movie.douban.com/chart'
    html=session.get(movie_url,headers=headers)
    soup=BeautifulSoup(html.text,'html.parser')
    result=soup.find_all('a',{'class':'nbg'})
    print(result)

# #爬取短评论
# def get_comment(filename):  #filename为爬取得内容保存的文件
#     begin=1
#     next_url='?start=20&limit=20&sort=new_score&status=P'
#     f=open(filename,'w+',encoding='utf-8')
#     while(True):
#         time.sleep(5)
#         comment_url='https://movie.douban.com/subject/26363254/comments'
#         data={
#             'start':'27',
#             'limit':'-20',
#             'sort':'new_score',
#             'status':'P'
#         }
#         headers2 = {
#             "Host": "movie.douban.com",
#             "Referer": "https://www.douban.com/",
#             'User-Agent': agent,
#             'Connection': 'keep-alive',
#         }

#         html=session.get(url='https://movie.douban.com/subject/26363254/comments'+next_url,headers=headers2)
#         soup=BeautifulSoup(html.text,'html.parser')

#         #爬取当前页面的所有评论
#         result=soup.find_all('div',{'class':'comment'}) #爬取得所有的短评
#         pattern4 = r'<p class=""> (.*?)' \
#                    r'</p>'
#         for item in result:
#             s=str(item)
#             count2=s.find('<p class="">')
#             count3=s.find('</p>')
#             s2=s[count2+12:count3]  #抽取字符串中的评论
#             if 'class' not in s2:
#                 f.write(s2)

#         #获取下一页的链接
#         next_url=soup.find_all('div',{'id':'paginator'})
#         pattern3=r'href="(.*?)">后页'
#         if(len(next_url)==0):
#             break
#         next_url=re.findall(pattern3,str(next_url[0]))  #得到后页的链接
#         if(len(next_url)==0): #如果没有后页的链接跳出循环
#             break
#         next_url=next_url[0]
#         print('%d爬取下一页评论...'%begin)
#         begin=begin+1
#         #如果爬取了6次则多休息2秒
#         if(begin%6==0):
#             time.sleep(30)
#             print('休息...')
#         print(next_url)
#     f.close()


#多线程爬虫，爬取豆瓣影评
def thread_get_comment(comment_url):
    next_url = '?start=19&limit=20&sort=new_score&status=P'
    headers2 = {
        "Host": "movie.douban.com",
        "Referer": "https://www.douban.com/",
        'User-Agent': agent,
        'Connection': 'keep-alive',
    }
   # f = open(filename, 'w+', encoding='utf-8')
   # comment_url = 'https://movie.douban.com/subject/26363254/comments'
    crawl_queue=[comment_url]
    #crawl_queue.append(comment_url+'start=480&limit=20&sort=new_score&status=P&percent_type=')
    #crawl_queue.append(comment_url+'?start=120&limit=20&sort=new_score&status=P&percent_type=')
    seen=set(crawl_queue)

    def process_queue():
        begin = 1
        while True:
            try:
                url=crawl_queue.pop()
            except  IndexError:
                break
            else:
                time.sleep(5)
                #,proxies={'https':'116.231.35.5:8118'}
                html = session.get(url=url,headers=headers2,proxies={'http':'218.56.132.156:8080'})
                #print (html.text)
                final=[]
                soup = BeautifulSoup(html.text, 'html.parser')
                #bs4 = BeautifulSoup(html.text, 'html.parser').body.find(class_='sub_ins')#找到评论区
                bs4 = soup.body.find('div',{'id':'comments'})#找到评论
                #print (bs4)
                
                comment_lists = bs4.find_all('div',class_='comment')
                for comment in comment_lists:
                    temp = []
                    info = comment.find('span',class_='comment-info')  # 获取用户和评分信息栏
                    username = [text for text in info.find('a').stripped_strings][0] # 获取用户名，去掉span！！
                    userid=info.find('a').get('href')
                    comment_time=info.find('span',class_='comment-time').get('title')
                    data = comment.find('p')
                    #12print (data)
#                     _time = data[0].get_text()  # 获取用户评论时间
#                     _time = re.search(r'[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]', _time)  # 正则表达
                    if info.find(class_=re.compile("allstar")) is not None:  # 找到评分栏
                        temp.append(username)  # 添加用户信息
                        #print (username)
                        #temp.append(city)
                        temp.append(userid)
                        temp.append(comment_time)
                        grade = info.find(class_=re.compile("allstar")).get('title')  # 获取用户评分
                        temp.append(grade)
                        temp.append(gradeDic[grade])  # 将评分转为5分制
                        print(grade)
                    else:
                        print("没有找到评分")
                        continue
                    if len(data) > 0:  # 如果用户评论不为空
                        comment_s = data.text  # 获取用户评论
                        temp.append(comment_s)
                        #print(comment_s)
                    else:
                        #print("用户未评论")
                        temp.append("")  # 留空
                    final.append(temp)
                write_data(final,'三生三世十里桃花.csv')
            
                next_url = soup.find_all('div', {'id': 'paginator'})
                #next_url = bs4.find('div', {'id': 'paginator'}).find('a',class_='next').get('href')
               # print (next_url)
                pattern3 = r'href="(.*?)">后页'
                if (len(next_url) == 0):
                    break
                next_url = re.findall(pattern3, str(next_url[0]))  # 得到后页的链接
                if (len(next_url) == 0):  # 如果没有后页的链接跳出循环
                    break
                next_url = next_url[0]
                print('%d爬取下一页评论...' % begin)
                begin = begin + 1
                # 如果爬取了6次则多休息2秒
                if (begin % 6 == 0):
                    print('休息...')
                    time.sleep(30)

                print(next_url)
                if comment_url+next_url not in seen:
                    seen.add(comment_url+next_url)
                    crawl_queue.append(comment_url+next_url)

    threads=[]
    max_threads=5
    while threads or crawl_queue:
        for thread in threads:
            if not thread.is_alive():
                threads.remove(thread)
        while len(threads)< max_threads and crawl_queue:
            thread=threading.Thread(target=process_queue)
            print('------------------------下一个线程----------')
            thread.setDaemon(True) # set daemon so main thread can exit when receive ctrl + C
            thread.start()
            threads.append(thread)
        time.sleep(2)


#    f.close()

if __name__=='__main__':
    if isLogin():
        print('您已经登录')
    else:
        print('xs')
        #login('_nafddgme','ffggpsd')
        login('15300523175','@Zy107868')
    #get_comment('key.txt')  #单线程爬虫
    #comment_url = 'https://movie.douban.com/subject/26363254/comments
    thread_get_comment('https://movie.douban.com/subject/25823277/comments')  #多线程爬虫
    #save_jieba_result('key2.txt')
    #draw_wordcloud('pjl_jieba.txt')



成功加载cookie
xs
['0ZPZByeQysrB4i3Qo7FW8sL8:en']
验证码连接 ['https://www.douban.com/misc/captcha?id=0ZPZByeQysrB4i3Qo7FW8sL8:en&amp;size=s']
获取验证码 https://www.douban.com/misc/captcha?id=0ZPZByeQysrB4i3Qo7FW8sL8:en&amp;size=s
test
please input the captcha
>glass
{'source': 'index_nav', 'form_email': '15300523175', 'form_password': '@Zy107868', 'captcha-solution': 'glass', 'captcha-id': '0ZPZByeQysrB4i3Qo7FW8sL8:en'}
登录成功
------------------------下一个线程----------
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
很差
1爬取下一页评论...
?start=20&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
很差
很差
很差
很差
很差
很差
很差
很差
推荐
还行
推荐
很差
很差
较差
推荐
力荐
很差
推荐
较差
很差
2爬取下一页评论...
?start=40&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
推荐
推荐
很差
很差
很差
还行
推荐
推荐
推荐
推荐
较差
推荐
力荐
力荐
很差
推荐
很差
推荐
较差
推荐
3爬取下一页评论...
?start=60&amp;limit=20&amp;sort=new_score&amp;status=P&amp;percent_type=
还行
没有找到评分
很差
力荐
很差
推荐
较差
很差
很差
很差
很差
很差
很差
很差
很差
较差
推荐
没有找到评分
很差
较差
4爬取下一页评论...
?start=80&amp;limit=20&amp;s